<a href="https://colab.research.google.com/github/bgalbraith/accelerated-othello/blob/main/CUDA_enhanced_Othello_Rollouts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This is an initial pass at implementing massively parallel support for MCTS experiments using the game Othello.

It is based on custom CUDA kernels implemented via Numba's CUDA JIT capabilities.

Currently, the code does the following:
Given a number of games to evaluate in parallel, a starting board configuration, and the player who's turn it is, play all the games out to completion by taking random actions for each player. The final output is a vector indicating which player won each game or if it was a draw.

In [1]:
import os

from numba import jit, cuda, float32, int32
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
import numpy as np

## Setting up Numba for CUDA

In order for Numba to JIT compile CUDA kernels, it needs to know where the appropriate system libraries are. This is done using environment variables. The below values should be safe for Google Colab, but if an error occurs when attempting to compile, running the following in a cell will give you an idea of which paths to use:
```
!find / -iname 'libdevice'
!find / -iname 'libnvvm.so'
```

In [2]:
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/local/cuda-10.0/nvvm/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so"

We set the number of games as a constant here. We also use a ray-casting strategy for finding valid moves in Othello, so we establish the eight possible ray directions here as well. Note the dtype is set to `np.int32`. This is because we will be loading this onto the GPU later, and it's better to use 32-bit data types (`int32`, `float32`) for GPU ops unless you really need the higher precision.

In [3]:
N_GAMES = 32
RAYS = np.array([[0, 1],  # east
                 [0, -1],  # west
                 [1, 0],  # south
                 [-1, 0],  # north
                 [1, 1],  # southeast
                 [1 , -1],  # southwest
                 [-1, 1],  # northeast
                 [-1, -1]  # northwest
                 ], dtype=np.int32)

## CUDA Device Function: Ray Casting
A device function is a utility function meant to be called by CUDA other kernels. It doesn't have implicit access to the thread that is calling it, but it is allowed to return a value (kernels must return None).

In this case, the ray casting function is implemented as a device function as we need to use it in two different kernel calls. Its purpose is to determine if a ray cast in a particular direction results in a valid hit -- that there is a continuous line of opposing player tokens that ends in the current player's token.

In [4]:
@cuda.jit(device=True)
def ray_cast(board, x0, y0, ray, player):
  opponent = 3 - player
  x = x0 + ray[0]
  y = y0 + ray[1]
  
  if x < 0 or y < 0 or x >= 8 or y >= 8 or board[x, y] != opponent:
    return False
    
  x += ray[0]
  y += ray[1]
  while x >= 0 and y >= 0 and x < 8 and y < 8:
    if board[x, y] == 0:
      return False
    
    if board[x, y] == player:    
      return True
    
    x += ray[0]
    y += ray[1]

  return False

## CUDA Kernel: Find Action Probabilities
The first part of a player's turn is to find all the valid actions available to it. In the single-threaded solution, candidate positions were selected and then each one was sequentially checked by casting rays to determine if any resulted in a hit.

**A note on CUDA kernels**: As these functions cannot return values directly, we have to pass in references to arrays that we can write results to and then refer to later in the processing pipeline. In this case, the output array is called `action_probs` for reasons we will get to shortly.

In the CUDA SIMT model, we are going to have each thread evaluate a single position in each game by setting our thread grid to have n_blocks = N_GAMES and each block be an 8x8 array of threads. Here we just have each thread cast rays from its assigned spot. If the current spot isn't free, that thread can stop immediately and store a 0 for that position. If we do get a ray hit, we store a positive value for that position and stop. Finally, if no hits are observed, we also store a 0.

### Why action probabilities and not just booleans?

Choosing a random element from a list is trivial with numpy but is not trivial inside a CUDA kernel for a few reasons. First, to get the most speedup, you need to avoid transferring data between the GPU and host, so doing as much as possible in kernels is desirable. Second, we have multiple threads all executing concurrently but nondeterministic in time, so some kind of coordinated reduction strategy is needed to first isolate valid actions and then choose amongst them. The strategy here is to make that choice random, which adds additional complexity as now we need a source for RNG and a way to use that efficiently to pick an option. The approach I have come up with is to assign each position a probability score given by

$$\mathbb{1}_{[hit]} \cdot  p(position|board), p \sim \mathcal{U}(0,1).$$

We can then use a greedy strategy (i.e., an argmax) to select our random action, which we shall see in the next kernel function.

In [5]:
@cuda.jit
def find_actions(rng_states, boards, player, action_probs):
  rays = cuda.const.array_like(RAYS)

  tx = cuda.threadIdx.x
  ty = cuda.threadIdx.y
  bx = cuda.blockIdx.x
  tid = cuda.grid(1)
  
  # space isn't empty, invalid move
  if boards[bx, tx, ty] != 0:
    action_probs[bx, tx, ty] = 0
    return

  opponent = 3 - player
  for i in range(8):
    hit = ray_cast(boards[bx], tx, ty, rays[i], player)
    if hit:
      action_probs[bx, tx, ty] = xoroshiro128p_uniform_float32(rng_states, tid)
      return

  action_probs[bx, tx, ty] = 0

## CUDA Kernel: Select Action and Execute Step

This kernel is a bit involved as it's actually two kernels put together - a kernel implementing the argmax action selection strategy and a kernel to execute the action and update the game state. I chose to fuse them as 1) both have the same thread grid signature (n_grids = N_GAMES, n_threads = 8) and 2) the output of the action selection kernel immediately fed into the step execution kernel. This results in removing an unnecessary array allocation and reduces the number of kernel invocations.

### Argmax reduction and thread block shared memory

We don't have access to an out-of-the-box argmax function with Numba, so we need to roll our own. We take advantage of CUDA's shared memory feature which allows faster access to shared memory within a thread block. The approach here is to use all 8 threads to scan along the rows of the `action_probs` array and collect both the max value and max value index, then store those in shared memory. We then need to sync the threads to make sure they are all done before continuing. Next, we use just one thread to scan over these results and pick the max of the maxes. Once we get our final x and y position for our action, we store that in shared memory. We have another thread sync barrier to make sure the threads not computing the final action wait until it's available.

### Game state update

Finally, each thread retrieves the action. In the event no action is possible, we assign the action a value of (-1,-1) and exit out, but not before indicating the `game_status` array that the player had to pass (0 = played, 1 = passed). If we can play, each thread evaluates one ray cast and updates the board accordingly. Finally, exit out, setting the `game_status` to played.

In [6]:
@cuda.jit
def select_and_step(action_probs, boards, player, player_status):
  rays = cuda.const.array_like(RAYS)
  
  row_max = cuda.shared.array(shape=(8,), dtype=float32)
  row_max_idx = cuda.shared.array(shape=(8,), dtype=int32)
  actions = cuda.shared.array(shape=(2,), dtype=int32)
  
  tx = cuda.threadIdx.x  
  bx = cuda.blockIdx.x

  opponent = 3 - player

  # argmax reduction part 1
  current_max = 0
  current_idx = 0
  for i in range(8):
    if action_probs[bx, tx, i] > current_max:
      current_max = action_probs[bx, tx, i]
      current_idx = i
  
  row_max[tx] = current_max
  row_max_idx[tx] = current_idx
  cuda.syncthreads()
  
  # argmax reduction part 2
  if tx == 0:
    current_max = 0
    current_idx = 0
    for i in range(8):
      if row_max[i] > current_max:
        current_max = row_max[i]
        current_idx = i
    
    x_idx = -1
    y_idx = -1
    if current_max > 0:
      x_idx = current_idx
      y_idx = row_max_idx[current_idx]
    
    actions[0] = x_idx
    actions[1] = y_idx
  cuda.syncthreads()

  # execute action and update game state
  act_x = actions[0]
  act_y = actions[1]

  if act_x == -1:
    if tx == 0:
      player_status[bx][player-1] = 1
    return

  r = rays[tx]
  hit = ray_cast(boards[bx], act_x, act_y, r, player)
  if hit:
    x = act_x + r[0]
    y = act_y + r[1]
    while boards[bx, x, y] == opponent:
      boards[bx, x, y] = player
      x += r[0]
      y += r[1]
  
  if tx == 0:
    boards[bx, act_x, act_y] = player    
    player_status[bx][player-1] = 0

In addition to the kernels and device functions, we need to allocate arrays on the device for processing and storage. Here we have two for tracking the states of all the games (board configuration and player status) and one for handling RNG. We also have our intermediate storage for our action probabilities. These all get allocated directly on the device.

In [7]:
# state
boards = cuda.device_array(shape=(N_GAMES, 8, 8), dtype=np.float32)
player_status = cuda.device_array(shape=(N_GAMES, 2), dtype=np.int32)
rng_states = create_xoroshiro128p_states(8*8*N_GAMES, seed=1)

# intermediates
action_probs = cuda.device_array(shape=(N_GAMES, 8, 8), dtype=np.float32)

## Rollouts: Putting it all together

Finally, we have our rollout function which handles moving data on and off the device, invoking the kernels, and deciding when we're finished. This method takes in two values: the current state of the game board and which player's turn is next. We then make `N_GAMES` copies of the board and load it into our device array along with initializing player status. We then run all games to completion, which we infer by checking that both payers have passed in all games. This does mean that we are continuing to play games that have ended, but 1) the state is effectively frozen so the outcome will never change and 2) the cost of that extra computation is negligible in this parallel scenario. We then run our two kernels and swap players. Note also that we do have to copy this data from the GPU every turn in order to check it.

Finally, once all games have ended, we copy the data back to the host, count up each player's tiles and identify the winner, while accounting for possible ties.

While not implemented here, it is straightforward to pull off the game states and action probabilities if there was a need to retain the history of the games as well without incurring too much penalty.

In [8]:
def rollout(board, planner):  
  cuda.to_device(np.tile(board, (N_GAMES, 1, 1)), to=boards)
  cuda.to_device(np.zeros((N_GAMES, 2), dtype=np.int32), to=player_status)

  player = planner

  while np.sum(player_status.copy_to_host()) < 2*N_GAMES:
    find_actions[N_GAMES, (8,8)](rng_states, boards, player, action_probs)    
    select_and_step[N_GAMES, 8](action_probs, boards, player, player_status)    
    player = 3 - player

  results = boards.copy_to_host().astype(int)
  scores = np.array([np.bincount(b.flatten(), minlength=3)[1:] 
                     for b in results])
  winner = np.argmax(scores, axis=1) + 1
  winner[scores[:,0] == scores[:,1]] = 0

  return winner

Here we can try running the actual rollouts. The first time this is run, there is a slight delay as the code gets JIT compiled. Even so, it is very fast and doesn't see much difference in timing between 32 games and 32k games!

In [ ]:
# initial board layout
board = np.zeros((8, 8), dtype=np.float32)
board[[3, 4], [3, 4]] = 2
board[[3, 4], [4, 3]] = 1

winner = rollout(board, 1)
winner

### Was that action probability strategy really necessary?

Honestly, given the small size of the task, it would have been way simpler to have just pulled the valid action array off the GPU and then used numpy to randomly select one of them. This could have then been provided as an argument to the state execution step, and likely would have not seen a major drop in performance (I haven't checked). However, that wasn't as fun an exercise as it wouldn't have required working with RNG generation, shared memory, and solving the action selection problem in parallel ;)